In [ ]:
from datetime import datetime, timedelta
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
%matplotlib inline
%config IPCompleter.greedy = True
%config InlineBackend.figure_format = 'retina'
import numpy as np
import matplotlib.pyplot as plt
import cbpro
import os
from pathlib import Path
import seaborn as sns
import requests
from creds import api_key

os.environ["TF_GPU_THREAD_MODE"] = "gpu_private"

import tensorflow as tf
import tensorflow_probability as tfp
from pandasgui import show

tf.distribute.OneDeviceStrategy(device="/gpu:0")
policy = tf.keras.mixed_precision.Policy("mixed_float16")
tf.keras.mixed_precision.experimental.set_policy(policy)

public_client = cbpro.PublicClient()

In [ ]:
class MinerMeta(type):
    # def __init__(self):
    #     result = getattr(self, "df", None)
    #     if result is None:
    #         self.df = self.compile_historic(read_csv=True)

    def compile_historic(self, num_days=100, write_csv=False, read_csv=False):
        file = Path.cwd() / f"{self.coin}_histdata.csv"
        if read_csv is True:
            df = pd.read_csv(file, index_col="time", infer_datetime_format=True)
            return df
        else:
            finish = datetime.now()
            start = finish - timedelta(num_days)
            delta = timedelta(hours=300)
            df = pd.DataFrame()

            while finish > start:
                historic = public_client.get_product_historic_rates(
                    f"{self.coin}-USD",
                    granularity=3600,
                    start=start,
                    end=start + delta,
                )
                start += delta
                df = df.append(historic, ignore_index=True, verify_integrity=True)
            df.columns = ["time", "low", "high", "open", "close", "volume"]

            # timestamp_s = date_time.map(pd.Timestamp.timestamp)
            # day = 24 * 60 * 60
            # year = (365.2425) * day
            # df["Day sin"] = np.sin(timestamp_s * (2 * np.pi / day))
            # df["Day cos"] = np.cos(timestamp_s * (2 * np.pi / day))
            # df["Year sin"] = np.sin(timestamp_s * (2 * np.pi / year))
            # df["Year cos"] = np.cos(timestamp_s * (2 * np.pi / year))

            df["time"] = pd.to_datetime(df["time"], unit="s")
            df.set_index("time", inplace=True, verify_integrity=False)
            df.sort_index(ascending=False, inplace=True)

            if write_csv is True:
                df.to_csv(file, index=True)

            return df

    def get_day_stats(self):
        result = getattr(self, "day_stats", None)
        if result is None:
            ticker = public_client.get_product_24hr_stats(f"{self.coin}-USD")
            df = pd.DataFrame.from_dict(ticker, orient="index")
            self.day_stats = df
        return df

    def year_day_fft(self, col):
        df = self.compile_historic()
        today = datetime.today()
        ylim = int(df[col].max())
        plt.figure(figsize=(15, 10))

        fft = tf.signal.rfft(df[col])
        f_per_dataset = np.arange(0, len(fft))

        n_samples_h = 1
        hours_per_year = 24 * 365.2524
        hours_per_week = 24 * 7
        years_per_dataset = n_samples_h / (hours_per_year)
        hours_per_dataset = n_samples_h / (hours_per_week)

        f_per_year = f_per_dataset / years_per_dataset
        f_per_week = f_per_dataset / hours_per_dataset
        plt.step(f_per_week, np.abs(fft))
        plt.xscale("log")
        # plt.ylim(1000, ylim)
        # plt.xlim([0.1, max(plt.xlim())])
        plt.xticks([1, 7], labels=["1/Week", "1/day"])
        plt.xlabel("Frequency (log scale)")

        return plt.show()

    def ttsplit_norm(self, df, split_time=0.7, feature_plot=False):
        # train_df Test Split
        n = len(df)
        train_df = df[0 : int(n * 0.7)]
        val_df = df[int(n * 0.7) : int(n * 0.9)]
        test_df = df[int(n * 0.9) :]
        # Normalize the Data
        train_df_mean = train_df.mean()
        train_df_std = train_df.std()

        train_df = (train_df - train_df_mean) / train_df_std
        val_df = (val_df - train_df_mean) / train_df_std
        test_df = (test_df - train_df_mean) / train_df_std

        # Create Feature Plot if wanted
        if feature_plot is True:
            df_std = (df - train_df_mean) / train_df_std
            df_std = df_std.melt(var_name="Column", value_name="Normalized")
            plt.figure(figsize=(12, 6))
            ax = sns.violinplot(x="Column", y="Normalized", data=df_std)
            ax.set_xticklabels(df.keys(), rotation=90)
            ax.set_title("train_dfing Data Feature Dist with whole DF Mean")

            return train_df, val_df, test_df

        return train_df, val_df, test_df

    def __call__(self, *args, **kwargs):

        cls = type.__call__(self, *args)

        # setattr(cls, "compile_historic", self.compile_historic)
        # setattr(cls, "year_day_fft", self.year_day_fft)
        # setattr(cls, "ttsplit_norm", self.ttsplit_norm)
        # setattr(cls, "get_day_stats", self.get_day_stats)
        # setattr(cls, "day_stats", self.get_day_stats())

        # for key, value in historic.items():
        #     setattr(cls, "hist_" + key, value)
        # for key, value in ticker.items():
        #     setattr(cls, "tick_" + key, value)

        return cls


In [ ]:
# # api_key = r"ru1zaf0ssaa29394mb4ahp"
# lc_file = Path.cwd() / "lc_data.csv"
# coins = "ETH"


# def lc_data(read_csv=False):
#     file = Path.cwd() / "lunardata.csv"
#     if read_csv is True:
#         data = pd.read_csv(file, index_col="id")
#         return data
#     else:
#         assets_url = (
#             b"https://api.lunarcrush.com/v2?data=assets&key="
#             + api_key
#             + "&symbol="
#             + coins
#             + +"&data_points=720"
#         )
#         assets = json.loads(urllib.request.urlopen(assets_url).read())
#         # config = pd.DataFrame.from_dict(assets["config"], orient="index")
#         # usage = pd.DataFrame.from_dict(assets["usage"], orient="index")
#         data = pd.DataFrame.from_dict(assets["data"])
#         return data


# lc = lc_data()

# lc.tail()


In [3]:
# lc_file = Path.cwd() / "lc_data.csv"
# coins = "ETH"
# intervals = ["1d", "1w", "1m", "3m", "6m", "1y", "2y"]
# now = datetime.now()
# delta = timedelta(30)
# start = now - delta

# payload = {
#     "key": api_key,
#     "symbol": coins,
#     "change": intervals,
#     "data_points": "720",
#     # "start": datetime.timestamp(start),
#     # "end": datetime.timestamp(start + timedelta(30)),
# }

# r = requests.get("https://api.lunarcrush.com/v2?data=assets", params=payload)

# data = pd.DataFrame.from_dict(r.json()["data"][0])

# ts = data.timeSeries.to_dict()
# new = pd.DataFrame.from_dict(ts, orient="index")
# new.pop("asset_id")
# new["time"] = pd.to_datetime(new["time"], unit="s")
# new.set_index("time", inplace=True)
# new.sort_index(ascending=False)
# print("Min", new.index.min())
# print("Max", new.index.max())
# print(new.index.max() - new.index.min())
# print(list(new.columns))
# new.tail()


In [4]:
def compile_lc_data(coins="ETH"):
    intervals = ["1d", "1w", "1m", "3m", "6m", "1y", "2y"]
    now = datetime.now()
    # delta = timedelta(num_days_in_past)
    # ut = datetime.timestamp((now - delta))

    payload = {
        "key": api_key,
        "symbol": coins,
        "change": intervals,
        "data_points": "720",
        # "start": ut,
        # "end": datetime.timestamp(now),
    }

    r = requests.get("https://api.lunarcrush.com/v2?data=assets", params=payload)

    data = pd.DataFrame.from_dict(r.json()["data"][0])
    ts = data.timeSeries.to_dict()
    new = pd.DataFrame.from_dict(ts, orient="index")
    new.pop("asset_id")
    new["time"] = pd.to_datetime(new["time"], unit="s")
    new.set_index("time", inplace=True)
    new.sort_index(ascending=False)

    return new


df = compile_lc_data()
df.head()


,open,close,high,low,volume,market_cap,url_shares,unique_url_shares,reddit_posts,reddit_posts_score,...,medium,youtube,social_contributors,social_volume,price_btc,social_volume_global,social_dominance,market_cap_global,market_dominance,percent_change_24h
time,,,,,,,,,,,,,,,,,,,,,
2021-09-22 04:00:00,2871.693498,2847.833538,2877.445843,2838.193265,1.605492e+09,335588029679,806,238,51.0,248.0,...,NaN,NaN,2449,2333,0.067823,51310,4.546872,2.123960e+12,15.800107,-4.442331
2021-09-22 05:00:00,2848.539999,2877.774935,2896.275298,2848.539999,1.607638e+09,338185408773,721,346,39.0,232.0,...,NaN,NaN,2311,2617,0.068338,49220,5.316944,2.142619e+12,15.783743,-4.052719
2021-09-22 06:00:00,2877.909481,2934.582087,2937.045202,2862.570033,1.797744e+09,344411713237,573,350,39.0,122.0,...,NaN,NaN,2706,3036,0.069178,49093,6.184181,2.165891e+12,15.901617,-3.555070
2021-09-22 07:00:00,2934.974461,2947.575924,2964.320426,2920.040630,2.094381e+09,346634464265,743,396,33.0,185.0,...,NaN,NaN,2712,3709,0.069135,44081,8.414056,2.177553e+12,15.918533,-4.200274
2021-09-22 08:00:00,2947.692914,2931.191142,2958.718221,2926.151324,1.469603e+09,344445042042,852,378,45.0,193.0,...,NaN,1.0,2734,3174,0.069166,38208,8.307161,2.164796e+12,15.911200,-4.824285


In [11]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=.2, train_size=.8)

576
145


In [8]:
CONV_WIDTH = 3
multi_val_performance = {}
multi_performance = {}
dev_model = tf.keras.models.Sequential(
    [
        # tf.keras.layers.Lambda(lambda x: x[:, -CONV_WIDTH:, :]),
        tf.keras.layers.Conv1D(
            filters=64,
            kernel_size=(CONV_WIDTH),
            strides=1,
            padding="causal",
            activation="relu",
        ),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
        # tf.keras.layers.Bidirectional(
        #     tf.keras.layers.LSTM(64, return_sequences=True)),
        tf.keras.layers.Dense(128),
        # tf.keras.layers.Dropout(.2),
        tf.keras.layers.Dense(num_features),
    ]
)

history = compile_and_fit(dev_model, multi_window)

multi_val_performance["DEV"] = dev_model.evaluate(multi_window.val)
multi_performance["DEV"] = dev_model.evaluate(multi_window.test, verbose=0)
multi_window.plot(dev_model)
